#Setup

In [ ]:
!pip3 install transformers
!pip3 install datasets
!pip3 install sentencepiece
!pip3 install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 32.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.5 MB/s 
     |████████████████████████████████| 596 kB 68.1 MB/s 
     |████████████████████████████████| 101 kB 13.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 35.9 MB/s 
     |████████████████████████████████| 115 kB 73.7 MB/s 
     |████████████████████████████████| 141 kB 71.9 MB/s 
     |████████████████████████████████| 212 kB 33.0 MB/s 
     |████████████████████████████████| 127 kB 46.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully

# Running the IndicNER Model

Let's try annotating some Indian language sentences and get the named entities

In [ ]:
# Import all the necessary classes and initialize the tokenizer and model.
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicNER")

model = AutoModelForTokenClassification.from_pretrained("ai4bharat/IndicNER")

In [ ]:
def get_predictions( sentence, tokenizer, model ):
  # Let us first tokenize the sentence - split words into subwords
  tok_sentence = tokenizer(sentence, return_tensors='pt')

  with torch.no_grad():
    # we will send the tokenized sentence to the model to get predictions
    logits = model(**tok_sentence).logits.argmax(-1)

    # We will map the maximum predicted class id with the class label
    predicted_tokens_classes = [model.config.id2label[t.item()] for t in logits[0]]

    predicted_labels = []

    previous_token_id = 0
    # we need to assign the named entity label to the head word and not the following sub-words
    word_ids = tok_sentence.word_ids()
    for word_index in range(len(word_ids)):
        if word_ids[word_index] == None:
            previous_token_id = word_ids[word_index]
        elif word_ids[word_index] == previous_token_id:
            previous_token_id = word_ids[word_index]
        else:
            predicted_labels.append( predicted_tokens_classes[ word_index ] )
            previous_token_id = word_ids[word_index]

    return predicted_labels

In [ ]:
# let us try with some example sentences here
sentence = 'लगातार हमलावर हो रहे शिवपाल और राजभर को सपा की दो टूक, चिट्ठी जारी कर कहा- जहां जाना चाहें जा सकते हैं'

predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )

for index in range(len(sentence.split(' '))):
  print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

लगातार	O
हमलावर	O
हो	O
रहे	O
शिवपाल	B-PER
और	O
राजभर	B-PER
को	O
सपा	B-ORG
की	O
दो	O
टूक,	O
चिट्ठी	O
जारी	O
कर	O
कहा-	O
जहां	O
जाना	O
चाहें	O
जा	O
सकते	O
हैं	O


In [ ]:
sentence = 'ಶರಣ್ ರ ನೀವು ನೋಡಲೇಬೇಕಾದ ಟಾಪ್ 5 ಕಾಮಿಡಿ ಚಲನಚಿತ್ರಗಳು'

predicted_labels = get_predictions(sentence=sentence,
                                   tokenizer=tokenizer,
                                   model=model
                                   )

for index in range(len(sentence.split(' '))):
  print( sentence.split(' ')[index] + '\t' + predicted_labels[index] )

ಶರಣ್	B-PER
ರ	O
ನೀವು	O
ನೋಡಲೇಬೇಕಾದ	O
ಟಾಪ್	O
5	O
ಕಾಮಿಡಿ	O
ಚಲನಚಿತ್ರಗಳು	O


#Naampadam Dataset

The _Naampadam_ Dataset is a large dataset for Named Entity Recognition in 11 Indian languages.  _Naampadam_ means "named entity" in Sanskrit.

In [ ]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

lang='as'

raw_datasets = load_dataset('ai4bharat/naamapadam', lang)

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset naamapadam downloaded and prepared to /root/.cache/huggingface/datasets/ai4bharat___naamapadam/as/1.0.0/ee2503e9587b9c2360e5009e586a9d49a25e315226ff65f9c6c2e05c1bc5c7d6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# let's now print how the Dataset looks like
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['words', 'ner'],
        num_rows: 10266
    })
    test: Dataset({
        features: ['words', 'ner'],
        num_rows: 51
    })
    validation: Dataset({
        features: ['words', 'ner'],
        num_rows: 52
    })
})

In [ ]:
raw_datasets.column_names

{'test': ['words', 'ner'],
 'train': ['words', 'ner'],
 'validation': ['words', 'ner']}

In [ ]:
# let's print an instance of dataset
idx=1000
rec=raw_datasets['train'][idx]
for w, t in zip(rec['words'],rec['ner']):
  print('{}\t{}'.format(w,t))


কেশ্ব	0
কৰা	0
ৱেব	0
সমল	0
(	0
{	0
$	0
amount	0
}	0
{	0
$	0
unit	0
}	0
)	0


In [ ]:
column_names = raw_datasets["train"].column_names
print(column_names)

features = raw_datasets["train"].features
print(features)

['words', 'ner']
{'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner': Sequence(feature=ClassLabel(num_classes=7, names=['O', 'B-PER', 'I-PER', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG'], id=None), length=-1, id=None)}


In [ ]:
text_column_name = "words"
label_column_name = "ner"

In [ ]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_list = features[label_column_name].feature.names

label_to_id = {label_list[i]: features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-LOC': 3, 'I-LOC': 4, 'B-ORG': 5, 'I-ORG': 6}


# Training an NER Model with the dataset

We have already seen how to get predictions from fine-tuned NER model. We will now use the pre-trained IndicBERT model and fine-tune it for NER task.

Let us download a pre-trained model and fine-tune it for the task of NER. We will have to use the `AutoModelForTokenClassification` class to fine-tune the model

**Load Pre-trained Model**

In [ ]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

**Tokenize all texts and align the labels with them**

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
train_dataset = raw_datasets["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset #0:   0%|          | 0/3 [00:00<?, ?ba/s]

Running tokenizer on train dataset #1:   0%|          | 0/3 [00:00<?, ?ba/s]

Running tokenizer on train dataset #2:   0%|          | 0/3 [00:00<?, ?ba/s]

Running tokenizer on train dataset #3:   0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
eval_dataset = raw_datasets["validation"]
eval_dataset = eval_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on Validation dataset",
)

Running tokenizer on Validation dataset #0:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #1:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #3:   0%|          | 0/1 [00:00<?, ?ba/s]

Running tokenizer on Validation dataset #2:   0%|          | 0/1 [00:00<?, ?ba/s]

**Create Data Collator, Metrics**

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

**Set Training Arguments**

In [ ]:
# args=TrainingArguments(output_dir='output_dir',max_steps=5)
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).



**Training**

In [ ]:
# Initialize our Trainer
# early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=2)
# args.metric_for_best_model = "f1"
# args.load_best_model_at_end = True
# args.evaluation_strategy = IntervalStrategy.STEPS
# args.eval_steps = args.save_steps
# args.greater_is_better = True

trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    # callbacks=[early_stopping_callback],
    args=args,
)

In [ ]:
trainer.args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_names=None,
label_smo

In [ ]:
train_result = trainer.train()
metrics = train_result.metrics

The following columns in the training set don't have a corresponding argument in `AlbertForTokenClassification.forward` and have been ignored: words, ner. If words, ner are not expected by `AlbertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 10266
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3852


Step,Training Loss
500,0.347800
1000,0.264100
1500,0.239100
2000,0.202600
2500,0.185900
3000,0.161100
3500,0.144900


Saving model checkpoint to output_dir/checkpoint-500
Configuration saved in output_dir/checkpoint-500/config.json
Model weights saved in output_dir/checkpoint-500/pytorch_model.bin
tokenizer config file saved in output_dir/checkpoint-500/tokenizer_config.json
Special tokens file saved in output_dir/checkpoint-500/special_tokens_map.json
Saving model checkpoint to output_dir/checkpoint-1000
Configuration saved in output_dir/checkpoint-1000/config.json
Model weights saved in output_dir/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in output_dir/checkpoint-1000/tokenizer_config.json
Special tokens file saved in output_dir/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to output_dir/checkpoint-1500
Configuration saved in output_dir/checkpoint-1500/config.json
Model weights saved in output_dir/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in output_dir/checkpoint-1500/tokenizer_config.json
Special tokens file saved in output_dir/checkpoint-15

In [ ]:

metrics = trainer.evaluate()

trainer.log_metrics("eval", metrics)

The following columns in the evaluation set don't have a corresponding argument in `AlbertForTokenClassification.forward` and have been ignored: words, ner. If words, ner are not expected by `AlbertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 52
  Batch size = 8


***** eval metrics *****
  epoch                   =        3.0
  eval_LOC_f1             =     0.7273
  eval_LOC_number         =          5
  eval_LOC_precision      =     0.6667
  eval_LOC_recall         =        0.8
  eval_ORG_f1             =     0.2703
  eval_ORG_number         =         19
  eval_ORG_precision      =     0.2778
  eval_ORG_recall         =     0.2632
  eval_PER_f1             =     0.6667
  eval_PER_number         =          3
  eval_PER_precision      =     0.6667
  eval_PER_recall         =     0.6667
  eval_loss               =     0.5214
  eval_overall_accuracy   =     0.8534
  eval_overall_f1         =     0.4074
  eval_overall_precision  =     0.4074
  eval_overall_recall     =     0.4074
  eval_runtime            = 0:00:01.83
  eval_samples_per_second =     28.371
  eval_steps_per_second   =      3.819


# Multilingual Fine-Tuning

We now present a short tutorial to fine-tune the model on the combined data of all Indic languages

The _Naampadam_ Dataset is a large dataset for Named Entity Recognition in 11 Indian languages.  _Naampadam_ means "named entity" in Sanskrit.

In [ ]:
# Let's download the Naampadam (Indic NER) dataset
from datasets import ClassLabel, load_dataset, load_metric, DownloadMode

languages=['as', 'bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'or', 'pa', 'ta', 'te']

# For demo purpose we will only choose 'Assamese' and 'Odiya' datasets
# languages=['as', 'or']

raw_datasets = {}

for lang in languages:
  raw_datasets[lang] = load_dataset('ai4bharat/naamapadam', lang,
                            use_auth_token='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',
                            download_mode=DownloadMode.FORCE_REDOWNLOAD)


We now concatenate all the datasets so that we could fine-tune a multilingual NER model

In [ ]:
# Let's look at how raw_dataset looks like
raw_datasets

In [ ]:
# If the labels are of type ClassLabel, they are already integers and we have the map stored somewhere.

label_column_name = 'ner'

label_list = raw_datasets['as']["train"].features[label_column_name].feature.names
label_to_id = {label_list[i]: raw_datasets['as']["train"].features[label_column_name].feature.str2int( label_list[i] ) for i in range(len(label_list))}

print(label_to_id)

num_labels = len(label_list)


To concatenate dataset let's combine all `train` and `validation` splits of all languages together

In [ ]:
pre_concatenated_train_split = []

for lang in raw_datasets:
  pre_concatenated_train_split.append( raw_datasets[lang]['train'] )

pre_concatenated_validation_split = []

for lang in raw_datasets:
  pre_concatenated_validation_split.append( raw_datasets[lang]['validation'] )

Let us concatenate the dataset now

In [ ]:
from datasets import concatenate_datasets, DatasetDict

concatenated_dataset = DatasetDict()
concatenated_dataset["train"] = concatenate_datasets(
    pre_concatenated_train_split
)

concatenated_dataset["validation"] = concatenate_datasets(
    pre_concatenated_validation_split
)

### Let us now load the Pre-trained model

In [ ]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np

config = AutoConfig.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels, finetuning_task='ner')
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/indic-bert")
model = AutoModelForTokenClassification.from_pretrained('ai4bharat/indic-bert', num_labels=num_labels )

In [ ]:
# Run the next cell if you want to use a GPU. Make sure that the Colab runtime is set accordingly

model=model.to("cuda")

Tokenize all the datasets and align them

In [ ]:
# Tokenize all texts and align the labels with them.
padding = "max_length"
text_column_name = 'words'
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples[text_column_name],
        padding=padding,
        truncation=True,
        max_length=512,
        # We use this argument because the texts in our dataset are lists of words (with a label for each word).
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples[label_column_name]):
        # print('=====')
        # print('{} {}'.format(i,label)) #ak
        word_ids = tokenized_inputs.word_ids(batch_index=i)

        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx

        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

Let us not `tokenize` the `train` and `validation` splits

In [ ]:
train_dataset = concatenated_dataset["train"]
train_dataset = train_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=32,
    load_from_cache_file=True,
    desc="Running tokenizer on train dataset",
)

In [ ]:
validation_dataset = concatenated_dataset["validation"]
validation_dataset = validation_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    num_proc=4,
    load_from_cache_file=True,
    desc="Running tokenizer on validation dataset",
)

## Create DataCollator and Metrics

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# Metrics
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                final_results[f"{key}_{n}"] = v
        else:
            final_results[key] = value
    return final_results

## Set Training Arguments

In [ ]:
args=TrainingArguments(
    output_dir='output_dir',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,)

In [ ]:
# Initialize the trainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    args=args,
)

## Train the model

In [ ]:
# Train the model
train_result = trainer.train()
metrics = train_result.metrics

## Evaluate the Trained Model

Let us now evaluate the trained model on the test sets of all languages

We need to first tokenize the test sets

In [ ]:
tokenized_test_set = {}

for lang in raw_datasets:
  tokenized_test_set[lang] = raw_datasets[lang]['test'].map(
      tokenize_and_align_labels,
      batched=True,
      num_proc=32,
      load_from_cache_file=True,
      desc="Running tokenizer on test dataset of language {0}".format(lang),
  )

Run prediction on test set of each of the language separately and extract overall `Precison`, `Recall` and `F-Score` separately

In [ ]:
final_metrics = {}

for lang in tokenized_test_set:
  predictions, labels, metrics = trainer.predict(tokenized_test_set[lang], metric_key_prefix=lang)

  lang_specific_results = {}
  for key in metrics:
    if 'overall_precision' in key:
      lang_specific_results['Precision'] = metrics[key]
    elif 'overall_recall' in key:
      lang_specific_results['Recall'] = metrics[key]
    elif 'overall_f1' in key:
      lang_specific_results['F1'] = metrics[key]
  final_metrics[lang] = lang_specific_results

Print the individual result on each of the language

In [ ]:
import pandas as pd

combined_results = pd.DataFrame.from_dict(
            final_metrics, orient="index"
        )

print(combined_results)

# Misc

In [ ]:
torch.__version__

In [ ]:
!nvidia-smi

In [ ]:
import torch
import gc
# del trainer
# del model
# del train_dataset
# del eval_dataset
# del data_collator
# del tokenizer
gc.collect()
torch.cuda.empty_cache()

from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)